# Isotropic and Anisotropic Kriging {#sec-iso-aniso-kriging}

This chapter illustrates the difference between isotropic and anisotropic Kriging models. The difference is illustrated with the help of the `spotPython` package. Isotropic Kriging models use the same `theta` value for every dimension. Anisotropic Kriging models use different `theta` values for each dimension.

## Example: Isotropic `Spot` Surrogate and the 2-dim Sphere Function


In [ ]:
import numpy as np
from math import inf
from spotPython.fun.objectivefunctions import analytical
from spotPython.spot import spot
from spotPython.utils.init import fun_control_init, surrogate_control_init
PREFIX="003"

### The Objective Function: 2-dim Sphere

* The `spotPython` package provides several classes of objective functions.
* We will use an analytical objective function, i.e., a function that can be described by a (closed) formula:
   $$f(x, y) = x^2 + y^2$$
* The size of the `lower` bound vector determines the problem dimension.
* Here we will use `np.array([-1, -1])`, i.e., a two-dim function.


In [ ]:
fun = analytical().fun_sphere
fun_control = fun_control_init(PREFIX=PREFIX,
                               lower = np.array([-1, -1]),
                               upper = np.array([1, 1]))

* Although the default `spot` surrogate model is an isotropic Kriging model, we will explicitly set the `theta` parameter to a value of `1` for both dimensions. This is done to illustrate the difference between isotropic and anisotropic Kriging models.


In [ ]:
surrogate_control=surrogate_control_init(n_theta=1)

In [ ]:
spot_2 = spot.Spot(fun=fun,
                   fun_control=fun_control,
                   surrogate_control=surrogate_control)

spot_2.run()

### Results


In [ ]:
spot_2.print_results()

In [ ]:
spot_2.plot_progress(log_y=True)

## Example With Anisotropic Kriging

* The default parameter setting of `spotPython`'s Kriging surrogate uses the same `theta` value for every dimension.
* This is referred to as "using an isotropic kernel". 
* If different `theta` values are used for each dimension, then an anisotropic kernel is used
* To enable anisotropic models in `spotPython`, the number of `theta` values should be larger than one.
* We can use `surrogate_control={"n_theta": 2}` to enable this behavior (`2` is the problem dimension).


In [ ]:
surrogate_control = surrogate_control_init(n_theta=2)
spot_2_anisotropic = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    surrogate_control=surrogate_control)
spot_2_anisotropic.run()

* The search progress of the optimization with the anisotropic model can be visualized:


In [ ]:
spot_2_anisotropic.plot_progress(log_y=True)

In [ ]:
spot_2_anisotropic.print_results()

In [ ]:
spot_2_anisotropic.surrogate.plot()

### Taking a Look at the `theta` Values

#### `theta` Values from the `spot` Model

* We can check, whether one or several `theta` values were used.
* The `theta` values from the surrogate can be printed as follows:


In [ ]:
spot_2_anisotropic.surrogate.theta

* Since the surrogate from the isotropic setting was stored as `spot_2`, we can also take a look at the `theta` value from this model:


In [ ]:
spot_2.surrogate.theta

#### TensorBoard

Now we can start TensorBoard in the background with the following command:





```{raw}
tensorboard --logdir="./runs"
```





We can access the TensorBoard web server with the following URL:





```{raw}
http://localhost:6006/
```





The TensorBoard plot illustrates how `spotPython` can be used as a microscope for the internal mechanisms of the surrogate-based optimization process. Here, one important parameter, the learning rate $\theta$ of the Kriging surrogate is plotted against the number of optimization steps.

![TensorBoard visualization of the spotPython surrogate model.](figures_static/03_tensorboard_03.png){width="100%"}



## Exercises


###  `fun_branin`

* Describe the function.
  * The input dimension is `2`. The search range is  $-5 \leq x_1 \leq 10$ and $0 \leq x_2 \leq 15$.
* Compare the results from `spotPython` run a) with isotropic and b) anisotropic surrogate models.
* Modify the termination criterion: instead of the number of evaluations (which is specified via `fun_evals`), the time should be used as the termination criterion. This can be done as follows (`max_time=1` specifies a run time of one minute):


In [ ]:
fun_evals=inf,
max_time=1,

### `fun_sin_cos`

* Describe the function.
  *  The input dimension is `2`. The search range is  $-2\pi \leq x_1 \leq 2\pi$ and $-2\pi \leq x_2 \leq 2\pi$.
* Compare the results from `spotPython` run a) with isotropic and b) anisotropic surrogate models.
* Modify the termination criterion (`max_time` instead of `fun_evals`) as described for `fun_branin`.

###  `fun_runge`

* Describe the function.
  *  The input dimension is `2`. The search range is  $-5 \leq x_1 \leq 5$ and $-5 \leq x_2 \leq 5$.
* Compare the results from `spotPython` run a) with isotropic and b) anisotropic surrogate models.
* Modify the termination criterion (`max_time` instead of `fun_evals`) as described for `fun_branin`.

###  `fun_wingwt`

* Describe the function.
  *  The input dimension is `10`. The search ranges are between 0 and 1 (values are mapped internally to their natural bounds).
* Compare the results from `spotPython` run a) with isotropic and b) anisotropic surrogate models.
* Modify the termination criterion (`max_time` instead of `fun_evals`) as described for `fun_branin`.




## Jupyter Notebook

:::{.callout-note}

* The Jupyter-Notebook of this lecture is available on GitHub in the [Hyperparameter-Tuning-Cookbook Repository](https://github.com/sequential-parameter-optimization/Hyperparameter-Tuning-Cookbook/blob/main/009_num_spot_anisotropic.ipynb)

:::
